In [ ]:
# default_exp losses

# losses

> API details.

In [ ]:
#export
from torch import nn
import torch

In [ ]:
#export
class VILoss(nn.Module):
    """
    Calculate the Kullback-Leibler divergence loss.
    Parameters
    ----------
    model : dies.embedding
        the given embedding to base the loss on
    lambd : float
        scalar for the loss
    """

    def __init__(
        self,
        model,
        base_loss=torch.nn.MSELoss(),
        kl_weight=0.1,
        scale_log_likelihood=True
    ):
        super(VILoss, self).__init__()
        self.base_loss = base_loss
        self.model = model
        self.kl_weight = kl_weight
        self.scale_log_likelihood=scale_log_likelihood

    def forward(self, y_hat, y):
        """

        Parameters
        ----------
        y : pytorch.Tensor
            any given tensor. Shape: [n, ]
        y_hat : pytorch.Tensor
            a tensor with the same shape as 'y'

        Returns
        -------
        pytorch.Tensor
            the resulting accumulated loss
        """
        base_loss = self.base_loss(y, y_hat)

        n_samples = max(len(y), 1)

        if self.scale_log_likelihood:
            base_loss = base_loss * n_samples

        kl = self.model.nn_kl_divergence()

        loss = base_loss + self.kl_weight * kl

        return loss

    def __repr__(self):
        s = "VILoss(\n  (base_loss):" + str(self.base_loss) + f"\n  (kl_weight): {self.kl_weight} \n)"
        return s

In [ ]:
#export
class Quantile_Score(torch.nn.Module):
    
    def reshape_1(self, x):
        return x.view(x.size()[0],1)
    
    def __init__(self, taus=[0.25, 0.5, 0.75]):
        super(Quantile_Score, self).__init__()
        
        self.taus =  torch.autograd.Variable(torch.tensor(taus, dtype=torch.float32), 
                                             requires_grad=False)
        
        self.taus = self.reshape_1(self.taus).t()
        
    def forward(self, y_hat, y):
        """
        Example:
            y = np.array([0.2, 0.1, 0.3, 0.4])
            tau=np.array([0.25,0.5,0.75])
            for each sample we have one row 
            y_hat = np.array([[0, 0.2, 0.3], 
                          [0.05, 0.1, 0.35], 
                          [0.2, 0.3, 0.6],
                          [0.3, 0.4, 0.45],])
            res = array([0.125 , 0.2   , 0.25  , 0.0625])
        """
        
        y = self.reshape_1(y)
        v = y - y_hat
        
        r = (torch.abs(v*(v>0).float()) * self.taus + \
             torch.abs(v*(v<0).float()) * (1-self.taus))

        # this would calculate the loss for each sample
        # r =  torch.sum(r,dim=1)
        r =  torch.sum(r)
        
        return r


In [ ]:
loss = Quantile_Score(taus=[0.25, 0.5, 0.75]).to("cpu")
probabilistic_forecasts = torch.tensor([[1,2,3],[4,5,6],[7,8,9],])
measurements = torch.tensor([2,5,8])
loss(probabilistic_forecasts, measurements)

tensor(1.5000)

In [ ]:
#export
class CnnMSELoss(torch.nn.MSELoss):
    def __init__(self):
        """
        Calculate the MSELoss and take the mean over all features
        """
        super(CnnMSELoss, self).__init__(None, None, "mean")

    def forward(self, y_hat, y):
        """

        Parameters
        ----------
        input : pytorch.Tensor
            any given tensor. Shape: [n, ]
        target : pytorch.Tensor
            a tensor with the same shape as 'input'

        Returns
        -------
        pytorch.Tensor
            the resulting loss
        """
        return torch.mean(torch.mean(torch.mean(torch.pow((y - yhat), 2), 2), 0))

In [ ]:
#export
class VAEReconstructionLoss(nn.Module):
    def __init__(self, model):
        """
        Calculate the sum of the Kullback–Leibler divergence loss and the loss of any given function
        Parameters
        ----------
        model : dies.autoencoder
            model of the autoencoder for which the loss is to be calculated
        """
        super(VAEReconstructionLoss, self).__init__()
        self.reconstruction_function = torch.nn.MSELoss()
        self.model = model

    def forward(self, x_hat, x):
        """

        Parameters
        ----------
        x : pytorch.Tensor
            any given tensor. Shape: [n, ]
        x_hat : pytorch.Tensor
            a tensor with the same shape as 'x'

        Returns
        -------
        pytorch.Tensor
            the resulting accumulated loss
        """
        recon_x = x_hat

        # how well do input x and output recon_x agree?

        generation_loss = self.reconstruction_function(recon_x, x)
        # KLD is Kullback–Leibler divergence -- how much does one learned
        # distribution deviate from another, in this specific case the
        # learned distribution from the unit Gaussian

        if self.model.embedding_module is None:
            mu, logvar = self.model.get_posteriors(x)
            # see Appendix B from VAE paper:
            # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
            # https://arxiv.org/abs/1312.6114
            # - D_{KL} = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
            # note the negative D_{KL} in appendix B of the paper
            KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
            # Normalise by same number of elements as in reconstruction
            KLD /= x.shape[0] * x.shape[1]

            # BCE tries to make our reconstruction as accurate as possible
            # KLD tries to push the distributions as close as possible to unit Gaussian
        else:
            KLD = 0
        return generation_loss + KLD


In [ ]:
# #export
# class GaussianNegativeLogLikelihoodLoss(nn.Module):
#     def __init__(self):
#         """
#         Calculate the Residual sum of squares loss
#         """
#         super(GaussianNegativeLogLikelihoodLoss, self).__init__()

#     def forward(self, y, mu_sigma):
#         """

#         Parameters
#         ----------
#         y : pytorch.Tensor
#             any given tensor. Shape: [n, ]
#         mu : pytorch.Tensor
#             a tensor with the same shape as 'y'
#         sigma : pytorch.Tensor
#             a tensor with the same shape as 'y'

#         Returns
#         -------
#         pytorch.Tensor
#             the resulting loss
#         """
#         mu, sigma = mu_sigma[:,0], mu_sigma[:,1]
#         nll =  torch.log(sigma)+(1/(2*sigma**2 ))*(y-mu)**2
#         nll =  nll.mean()
#         return nll

#export
class GaussianNegativeLogLikelihoodLoss(nn.Module):
    def __init__(self, num_mini_batches_mse=5):
        """
        Calculate the Residual sum of squares loss
        """
        super(GaussianNegativeLogLikelihoodLoss, self).__init__()
        self.loss = nn.GaussianNLLLoss(reduction="mean")
        self.num_mini_batches_mse = num_mini_batches_mse
        self.counter=0

    def forward(self, mu_sigma, y):
        """

        Parameters
        ----------
        y : pytorch.Tensor
            any given tensor. Shape: [n, ]
        mu : pytorch.Tensor
            a tensor with the same shape as 'y'
        sigma : pytorch.Tensor
            a tensor with the same shape as 'y'

        Returns
        -------
        pytorch.Tensor
            the resulting loss
        """
        y_pred, var_pred = mu_sigma[0].reshape(-1,1), mu_sigma[1].reshape(-1,1)
        
        _mse = (y - y_pred)**2
        
        N = y.shape[0]
        
        if self.counter % self.num_mini_batches_mse*2 < self.num_mini_batches_mse:
            loss = 1 / (N * 2 * var_pred.exp()) + var_pred/2
        else:
            loss = _mse
        
        loss = loss.mean()
        
        self.counter += 1
        
        return loss


In [ ]:
#export
class RSSLoss(nn.Module):
    def __init__(self):
        """
        Calculate the Residual sum of squares loss
        """
        super(RSSLoss, self).__init__()

    def forward(self, y, y_hat):
        """

        Parameters
        ----------
        y : pytorch.Tensor
            any given tensor. Shape: [n, ]
        y_hat : pytorch.Tensor
            a tensor with the same shape as 'y'

        Returns
        -------
        pytorch.Tensor
            the resulting loss
        """
        return ((y - y_hat) ** 2).sum()

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00a_utils.ipynb.
Converted 00b_losses.ipynb.
Converted 00c_utils_blitz.ipynb.
Converted 00d_baselines.ipynb.
Converted 00e_metrics.ipynb.
Converted 01_tabular.core.ipynb.
Converted 02_tabular.data.ipynb.
Converted 03_tabular.model.ipynb.
Converted 04_tabular.learner.ipynb.
Converted 05_timeseries.core.ipynb.
Converted 06_timeseries.data.ipynb.
Converted 07_timeseries.model.ipynb.
Converted 08_timeseries.learner.ipynb.
Converted 09_gan.core.ipynb.
Converted 10_autoencoder_models.ipynb.
Converted 11_probabilistic_models.ipynb.
Converted index.ipynb.
